In [ ]:
df = spark.table("default.tweets_20250623_001833")
df.printSchema()
df.show(truncate=False)

In [ ]:
%pip install textblob

In [ ]:
from textblob import TextBlob
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

# Define sentiment analysis function
def get_sentiment(text):
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return 0.0

# Register UDF
sentiment_udf = udf(get_sentiment, FloatType())

# Add sentiment score column
df_sentiment = df.withColumn("sentiment", sentiment_udf(df["text"]))
df_sentiment.select("text", "sentiment").show(truncate=False)

In [ ]:
df_sentiment.write.format("delta").mode("overwrite").save("dbfs:/FileStore/twitter/gold/tweets_sentiment")

# Optional: register as table
spark.sql("""
    CREATE OR REPLACE TABLE default.tweets_sentiment_gold
    USING DELTA
    LOCATION 'dbfs:/FileStore/twitter/gold/tweets_sentiment'
""")